In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
import numpy as np 


nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
text=pd.read_csv('Electronic_World.txt',delimiter='\t')
df=text['title'].tolist()

stemmer=PorterStemmer()
stopWords=set(stopwords.words('english'))

def preprocess(text):
    tokens=word_tokenize(text.lower())
    return " ".join([stemmer.stem(word) for word in tokens if word not in stopWords])
cleanedData=[preprocess(sentence) for sentence in df]

cleanedData=cleanedData[:20]
cleanedData

tokenizer=Tokenizer()

def getSequences(data):
    tokenizer.fit_on_texts(data)
    totalWords=len(tokenizer.word_index)+1
    inputSequences=[]
    for sentence in data:
        tokenList=tokenizer.texts_to_sequences([sentence])[0]
        for i in range(1,len(tokenList)):
            nGramSeq=tokenList[:i+1]
            inputSequences.append(nGramSeq)
    return inputSequences,totalWords
    
inputSequences,totalWords=getSequences(cleanedData)
# print(inputSequences)

def padder(inputs):
    maxLen=max([len(row) for row in inputs])
    inputs=pad_sequences(inputs,maxlen=maxLen,padding='pre')
    x,y=inputs[:,:-1],inputs[:,-1]
    y=to_categorical(y,num_classes=totalWords)
    return x,y,inputs,maxLen

x,y,inputs,maxLen=padder(inputSequences)

# print(y)
model=Sequential([
    Embedding(totalWords,10,input_length=maxLen),
    LSTM(50),
    Dense(totalWords, activation="softmax")
])
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x,y,epochs=5,verbose=1)


def output(outText,reqLen):
    for i in range(reqLen):
        seed=tokenizer.texts_to_sequences([outText])[0]
        seed=pad_sequences([seed],maxlen=4,padding='pre')
        preds=model.predict(seed)[0]
        index=np.random.choice(range(totalWords),p=preds)
        data=tokenizer.index_word.get(index)
        outText+=" "
        outText+=data
    return outText
print(output("Good Watch",4))




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naras\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\naras\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 5.3187
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.3143
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.3106
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.3044
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.2717
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Good Watch watch promis origin order
